In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
data_startup = pd.read_csv('50_Startups.csv')

In [3]:
data_startup.sample(7)

,R&D Spend,Administration,Marketing Spend,State,Profit
36,28663.76,127056.21,201126.82,Florida,90708.19
27,72107.60,127864.55,353183.81,New York,105008.31
25,64664.71,139553.16,137962.62,California,107404.34
11,100671.96,91790.61,249744.55,California,144259.40
34,46426.07,157693.92,210797.67,California,96712.80
4,142107.34,91391.77,366168.42,Florida,166187.94
1,162597.70,151377.59,443898.53,California,191792.06


In [ ]:
# 1. Lakukan analisa mengenai field mana saja yang memiliki data kosong(Nan) lalu Isi data yang kosong tersebut menggunakan nilai mean

In [18]:
1.
df = pd.DataFrame(data)
cols_with_missing = df.columns[df.isnull().any()].tolist()
for col in cols_with_missing:
    df[col] = df[col].fillna(df[col].mean())
print(df)

     R&D Spend  Administration  Marketing Spend       State         Profit
0   165349.200   136897.800000    471784.100000    New York  192261.830000
1   162597.700   151377.590000    443898.530000  California  191792.060000
2   153441.510   101145.550000    407934.540000     Florida  191050.390000
3   144372.410   118671.850000    383199.620000    New York  182901.990000
4   142107.340    91391.770000    366168.420000     Florida  166187.940000
5   131876.900    99814.710000    362861.360000    New York  156991.120000
6   134615.460   147198.870000    127716.820000  California  156122.510000
7   130298.130   122790.158298    323876.680000     Florida  155752.600000
8   120542.520   148718.950000    311613.290000    New York  152211.770000
9   123334.880   108679.170000    304981.620000  California  149759.960000
10   76788.862   110594.110000    229160.950000     Florida  146121.950000
11  100671.960    91790.610000    249744.550000  California  144259.400000
12   93863.750   127320.3

In [ ]:
# 2. lakukan oneHotEncoder ke field state

In [14]:
2.
df = pd.DataFrame(data)
df = pd.get_dummies(df, columns=['State'])
print(df)

    R&D Spend  Administration  Marketing Spend     Profit  State_California  \
0   165349.20       136897.80        471784.10  192261.83             False   
1   162597.70       151377.59        443898.53  191792.06              True   
2   153441.51       101145.55        407934.54  191050.39             False   
3   144372.41       118671.85        383199.62  182901.99             False   
4   142107.34        91391.77        366168.42  166187.94             False   
5   131876.90        99814.71        362861.36  156991.12             False   
6   134615.46       147198.87        127716.82  156122.51              True   
7   130298.13             NaN        323876.68  155752.60             False   
8   120542.52       148718.95        311613.29  152211.77             False   
9   123334.88       108679.17        304981.62  149759.96              True   
10        NaN       110594.11        229160.95  146121.95             False   
11  100671.96        91790.61        249744.55  1442

In [ ]:
# 3. buat field baru yaitu Tax. dimana Tax = (profit + Marketing Spend + administration) * 5%

In [16]:
3.
df = pd.DataFrame(data)
df['Administration'].fillna(df['Administration'].mean(), inplace=True)
df['Tax'] = (df['Profit'] + df['Marketing Spend'] + df['Administration']) * 0.05
print(df)

    R&D Spend  Administration  Marketing Spend       State     Profit  \
0   165349.20   136897.800000        471784.10    New York  192261.83   
1   162597.70   151377.590000        443898.53  California  191792.06   
2   153441.51   101145.550000        407934.54     Florida  191050.39   
3   144372.41   118671.850000        383199.62    New York  182901.99   
4   142107.34    91391.770000        366168.42     Florida  166187.94   
5   131876.90    99814.710000        362861.36    New York  156991.12   
6   134615.46   147198.870000        127716.82  California  156122.51   
7   130298.13   122790.158298        323876.68     Florida  155752.60   
8   120542.52   148718.950000        311613.29    New York  152211.77   
9   123334.88   108679.170000        304981.62  California  149759.96   
10        NaN   110594.110000        229160.95     Florida  146121.95   
11  100671.96    91790.610000        249744.55  California  144259.40   
12   93863.75   127320.380000        249839.44     

In [20]:
# 4. Jalankan standardScaler field-field tersebut

In [19]:
4.
df = pd.DataFrame(data)

# Menangani nilai yang hilang
imputer = SimpleImputer(strategy='mean')
df[['Administration', 'Marketing Spend']] = imputer.fit_transform(df[['Administration', 'Marketing Spend']])

# One-hot encoding untuk kolom "State"
df_encoded = pd.get_dummies(df, columns=['State'], drop_first=True)

# Pipeline untuk melakukan standarisasi dan preprocessing kolom numerik
numeric_features = ['R&D Spend', 'Administration', 'Marketing Spend']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

# Menggabungkan preprocessing dengan model
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Transformasi data
df_scaled = pipeline.fit_transform(df_encoded)

# Menggabungkan hasil transformasi dengan kolom target (Profit)
df_scaled = pd.DataFrame(df_scaled, columns=numeric_features)
df_scaled['State'] = df_encoded['State_Florida']  # Ambil kolom State Florida sebagai contoh
df_scaled['Profit'] = df['Profit']  # Kolom target

print(df_scaled)

    R&D Spend  Administration  Marketing Spend  State     Profit
0    1.969800    5.543175e-01         2.299248  False  192261.83
1    1.908599    1.123258e+00         2.040926  False  191792.06
2    1.704943   -8.504601e-01         1.707769   True  191050.39
3    1.503224   -1.618166e-01         1.478634  False  182901.99
4    1.452843   -1.233706e+00         1.320863   True  166187.94
5    1.225293   -9.027514e-01         1.290227  False  156991.12
6    1.286206    9.590672e-01        -0.888067  False  156122.51
7    1.190178   -5.717739e-16         0.929087   True  155752.60
8    0.973189    1.018794e+00         0.815484  False  152211.77
9    1.035298   -5.544490e-01         0.754050  False  149759.96
10        NaN   -4.792072e-01         0.051675   True  146121.95
11   0.531219   -1.218034e+00         0.242354  False  144259.40
12   0.379787    1.780015e-01         0.243233   True  141585.52
13   0.338164    4.992015e-01         0.269407  False        NaN
14   0.959859    1.326391